In [180]:
import os
import csv
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

result = list()
for file in os.listdir('kungalv_slutpriser'):
    file_path = os.path.join('kungalv_slutpriser', file)

    with open(file_path, 'r', encoding='utf-8') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')
    
    for i in soup.find_all(class_="sold-results__normal-hit"):
        
        dates = i.find('span', class_="hcl-label hcl-label--state hcl-label--sold-at")
        dates_of_sale = dates.text.replace('Såld','').strip() if dates else ''
        
        addresses = i.find('h2', class_="sold-property-listing__heading qa-selling-price-title hcl-card__title")
        address = addresses.text.strip() if addresses else ''
        
        locations = i.find('div', class_="sold-property-listing__location")
        text = locations.find('div').get_text(strip=True).replace("\n","").replace("VillaVilla","").replace(" ", "").strip()
        
        prices = i.find('span', class_="hcl-text")
        if 'Slutpris' in prices.string:
            line = str(prices.string).strip()
            parts = line.split(" ",1)
            closing_price = parts[1].replace('\xa0', '').replace('kr','')
        

        area = i.find('div', class_="sold-property-listing__subheading sold-property-listing__area").get_text().strip()
        area = area.replace('\xa0',' ').replace(" ", "").replace("\n","").replace("rum","").replace(',','.').strip()

        total_area = ''
        
        if area != '':   
            b_area = area.rsplit('m²')
            
            if len(b_area) > 1:
                rooms = b_area[1]
                rooms = rooms.replace(' rum','')
            elif 'rum' in b_area[0]:
                rooms = b_area[0]
                rooms = rooms.replace(' rum','')
            else:
                rooms = ''

            area = b_area[0]
            
            if '+' in area:
                split_area = area.rsplit('+')
                #area = split_area[0]
                total_area = float(split_area[0]) + float(split_area[1])
            #print(area)
    
        if i.find('div', class_="sold-property-listing__land-area"):
            plot = i.find('div', class_="sold-property-listing__land-area").get_text().strip()
            plot = plot.replace("m²","").replace("tomt","").replace('\xa0', '').strip() 
        else:
            plot = ''

            
        result.append([dates_of_sale, address, text, closing_price, area, total_area, rooms, plot])
        


data = list()

for obj in result:
    row = { 'Date of Sale' : obj[0],
            'Address' : obj[1],
            'Location' : obj[2],
            'Area': obj[4],
            'Total Area':obj[5],
            'Rooms': obj[6],
            'Plot': obj[7],
            'Closing Price' : obj[3]
          }
    data.append(row)
data = pd.DataFrame(data)
print(data)
data.to_csv('HousePrices1.csv',index=None)
 

          Date of Sale                    Address  \
0     23 november 2017                Sjöhåla 580   
1     18 november 2017             Galeasgatan 15   
2     17 november 2017        Västerhöjdsvägen 36   
3     16 november 2017           Gråstensvägen 19   
4     16 november 2017                 Hägnan 135   
...                ...                        ...   
1968     14 april 2016              Olvonvägen 71   
1969     14 april 2016            Rättarevägen 16   
1970     12 april 2016            Ametistvägen 11   
1971      9 april 2016  Rådalsvägen 25 Havsutsikt   
1972      8 april 2016               Annas Väg 10   

                              Location     Area Total Area Rooms    Plot  \
0            Kovikshamn,Kungälvskommun    94+87      181.0     5    1068   
1               Kungälv,Kungälvskommun   103+64      167.0     5     610   
2                 Kärna,Kungälvskommun      107                5     258   
3          KodeHalltorp,Kungälvskommun       94            